In [64]:
# 专业级中文PDF信息精确提取系统
!pip install pdfplumber jieba fasthanlan paddleocr transformers torch -qq
!pip install git+https://github.com/PaddlePaddle/PaddleOCR.git -q

import pdfplumber
import re
import jieba
import jieba.posseg as pseg
import pandas as pd
import numpy as np
from collections import Counter
from google.colab import files
import io
import torch
from transformers import BertTokenizer, BertForTokenClassification
import requests

print("=== 专业级中文PDF信息精确提取系统 ===")

# 高级初始化
jieba.initialize()

# 专业历史文献词典
PROFESSIONAL_DICT = {
    # 历史人物
    'person': [
        '张隐韬', '王金风', '李大钊', '毛泽东', '周恩来', '蒋介石', '孙中山', '袁世凯',
        '陈独秀', '胡适', '鲁迅', '梁启超', '蔡元培', '章太炎', '朱德', '刘少奇', '邓小平',
        '彭德怀', '贺龙', '刘伯承', '陈毅', '林彪', '宋庆龄', '冯玉祥', '张学良'
    ],
    # 历史地点
    'location': [
        '南皮县', '河北省', '天津市', '北京市', '上海市', '广州市', '武汉市', '南京市',
        '重庆市', '西安市', '延安', '井冈山', '瑞金', '遵义', '庐山', '黄埔', '北平', '南京'
    ],
    # 历史组织
    'organization': [
        '中国共产党', '国民党', '黄埔军校', '国民军', '八路军', '新四军', '解放军',
        '中央政府', '国务院', '政治局', '中央委员会', '共青团', '工会', '农会'
    ],
    # 历史事件
    'event': [
        '五四运动', '北伐战争', '南昌起义', '秋收起义', '抗日战争', '解放战争', '土地改革',
        '文化大革命', '改革开放', '第一次国共合作', '第二次国共合作', '西安事变',
        '卢沟桥事变', '百团大战', '辽沈战役', '淮海战役', '平津战役'
    ]
}

# 加载专业词典
for category, words in PROFESSIONAL_DICT.items():
    for word in words:
        jieba.add_word(word, freq=1000000, tag=category[:2])

class ProfessionalChineseNER:
    def __init__(self):
        self.surname_set = self._load_comprehensive_surnames()
        self.location_patterns = self._build_location_patterns()
        self.event_patterns = self._build_event_patterns()

    def _load_comprehensive_surnames(self):
        """加载完整姓氏库"""
        surnames = set()
        # 单姓
        single_surnames = '''
            赵钱孙李周吴郑王冯陈褚卫蒋沈韩杨朱秦尤许何吕施张孔曹严华金魏陶姜
            戚谢邹喻柏水窦章云苏潘葛奚范彭郎鲁韦昌马苗凤花方俞任袁柳酆鲍史唐
            费廉岑薛雷贺倪汤滕殷罗毕郝邬安常乐于时傅皮卞齐康伍余元卜顾孟平黄
            和穆萧尹姚邵湛汪祁毛禹狄米贝明臧计伏成戴谈宋茅庞熊纪舒屈项祝董梁
            杜阮蓝闵席季麻强贾路娄危江童颜郭梅盛林刁钟徐邱骆高夏蔡田樊胡凌霍
            虞万支柯昝管卢莫经房裘缪干解应宗丁宣贲邓郁单杭洪包诸左石崔吉钮龚
            程嵇邢滑裴陆荣翁荀羊於惠甄曲家封芮羿储靳汲邴糜松井段富巫乌焦巴弓
            牧隗山谷车侯宓蓬全郗班仰秋仲伊宫宁仇栾暴甘钭厉戎祖武符刘景詹束龙
            叶幸司韶郜黎蓟薄印宿白怀蒲邰从鄂索咸籍赖卓蔺屠蒙池乔阴鬱胥能苍双
            闻莘党翟谭贡劳逄姬申扶堵冉宰郦雍卻璩桑桂濮牛寿通边扈燕冀郏浦尚农
            温别庄晏柴瞿阎充慕连茹习宦艾鱼容向古易慎戈廖庾终暨居衡步都耿满弘
            匡国文寇广禄阙东欧殳沃利蔚越夔隆师巩厍聂晁勾敖融冷訾辛阚那简饶空
            曾毋沙乜养鞠须丰巢关蒯相查后荆红游竺权逯盖益桓公万俟司马上官欧阳
            夏侯诸葛闻人东方赫连皇甫尉迟公羊澹台公冶宗政濮阳淳于单于太叔申屠
            公孙仲孙轩辕令狐钟离宇文长孙慕容司徒司空
        '''
        for char in single_surnames:
            if char.strip():
                surnames.add(char)
        return surnames

    def _build_location_patterns(self):
        """构建地理位置识别模式"""
        return {
            'province': r'(河北|山西|辽宁|吉林|黑龙江|江苏|浙江|安徽|福建|江西|山东|河南|湖北|湖南|广东|海南|四川|贵州|云南|陕西|甘肃|青海|台湾)省?',
            'city': r'(北京|天津|上海|重庆|石家庄|太原|沈阳|长春|哈尔滨|南京|杭州|合肥|福州|南昌|济南|郑州|武汉|长沙|广州|南宁|海口|成都|贵阳|昆明|西安|兰州|西宁|台北)市?',
            'county': r'[\u4e00-\u9fa5]{1,6}[县旗]',
            'district': r'[\u4e00-\u9fa5]{1,6}[区盟]',
            'town': r'[\u4e00-\u9fa5]{1,6}[镇乡]',
            'village': r'[\u4e00-\u9fa5]{1,6}[村街坊]',
            'specific': r'(华北|华东|华南|华中|西北|西南|东北|京津冀|长三角|珠三角|长江|黄河|珠江|松花江)'
        }

    def _build_event_patterns(self):
        """构建事件识别模式"""
        return {
            'movement': r'[\u4e00-\u9fa5]{1,6}运动',
            'uprising': r'[\u4e00-\u9fa5]{1,6}起义',
            'war': r'[\u4e00-\u9fa5]{1,6}战争',
            'battle': r'[\u4e00-\u9fa5]{1,6}战役',
            'incident': r'[\u4e00-\u9fa5]{1,6}事件',
            'reform': r'[\u4e00-\u9fa5]{1,6}改革',
            'revolution': r'[\u4e00-\u9fa5]{1,6}革命',
            'meeting': r'[\u4e00-\u9fa5]{1,6}会议'
        }

    def precise_person_recognition(self, text):
        """精准人名识别"""
        persons = set()

        # 方法1: 专业词典匹配
        for person in PROFESSIONAL_DICT['person']:
            if person in text:
                persons.add(person)

        # 方法2: 基于姓氏的精确匹配
        for surname in self.surname_set:
            # 匹配 "姓氏+1-2字" 的模式
            pattern1 = surname + r'[\u4e00-\u9fa5]{1}'
            pattern2 = surname + r'[\u4e00-\u9fa5]{2}'

            matches1 = re.findall(pattern1, text)
            matches2 = re.findall(pattern2, text)

            for match in matches1 + matches2:
                if 2 <= len(match) <= 4:  # 限制合理的姓名长度
                    persons.add(match)

        # 方法3: jieba词性标注
        words = pseg.cut(text)
        for word, flag in words:
            if flag == 'nr' and 2 <= len(word) <= 4:
                # 验证是否包含常见姓氏
                if any(word.startswith(surname) for surname in self.surname_set):
                    persons.add(word)

        return sorted(list(persons))

    def precise_location_recognition(self, text):
        """精准地名识别"""
        locations = set()

        # 方法1: 专业词典匹配
        for location in PROFESSIONAL_DICT['location']:
            if location in text:
                locations.add(location)

        # 方法2: 模式匹配
        for pattern_type, pattern in self.location_patterns.items():
            matches = re.findall(pattern, text)
            for match in matches:
                if isinstance(match, tuple):
                    match = match[0]  # 处理分组匹配
                locations.add(match)

        # 方法3: 行政区划后缀匹配
        administrative_suffixes = ['省', '市', '县', '区', '镇', '乡', '村']
        for suffix in administrative_suffixes:
            pattern = r'[\u4e00-\u9fa5]{1,6}' + suffix
            matches = re.findall(pattern, text)
            locations.update(matches)

        return sorted(list(locations))

    def precise_organization_recognition(self, text):
        """精准组织识别"""
        organizations = set()

        # 方法1: 专业词典匹配
        for org in PROFESSIONAL_DICT['organization']:
            if org in text:
                organizations.add(org)

        # 方法2: 常见组织后缀
        org_suffixes = ['党', '政府', '委员会', '部队', '军队', '学校', '大学', '学院', '机关', '部门']
        for suffix in org_suffixes:
            pattern = r'[\u4e00-\u9fa5]{1,8}' + suffix
            matches = re.findall(pattern, text)
            organizations.update(matches)

        # 方法3: jieba识别
        words = pseg.cut(text)
        for word, flag in words:
            if flag == 'nt' and len(word) >= 2:
                organizations.add(word)

        return sorted(list(organizations))

    def precise_event_recognition(self, text):
        """精准事件识别"""
        events = set()

        # 方法1: 专业词典匹配
        for event in PROFESSIONAL_DICT['event']:
            if event in text:
                events.add(event)

        # 方法2: 事件模式匹配
        for pattern_type, pattern in self.event_patterns.items():
            matches = re.findall(pattern, text)
            events.update(matches)

        # 方法3: 时间+事件模式
        time_event_pattern = r'(\d{1,4}年\S{1,10}?(?:运动|起义|战争|战役|事件|改革))'
        matches = re.findall(time_event_pattern, text)
        events.update(matches)

        return sorted(list(events))

    def time_recognition(self, text):
        """时间信息识别"""
        times = set()

        # 年月日模式
        patterns = [
            r'\d{1,4}年\d{1,2}月\d{1,2}日',
            r'\d{1,4}年\d{1,2}月',
            r'\d{1,4}年',
            r'\d{1,2}月\d{1,2}日',
            r'[一二三四五六七八九十]+年',
            r'[春夏秋冬][季天]',
            r'(?:民国|公元|光绪|宣统|康熙|乾隆)\d{1,4}年'
        ]

        for pattern in patterns:
            matches = re.findall(pattern, text)
            times.update(matches)

        return sorted(list(times))

    def extract_with_context(self, text, entities):
        """提取实体上下文"""
        context_results = {}

        for entity_type, entity_list in entities.items():
            context_results[entity_type] = {}
            for entity in entity_list[:10]:  # 每个类型取前10个
                # 找到实体出现的句子
                sentences = re.split(r'[。！？!?]', text)
                for sentence in sentences:
                    if entity in sentence and len(sentence) > 10:
                        context_results[entity_type][entity] = sentence.strip()
                        break

        return context_results

def extract_text_advanced(pdf_content):
    """高级文本提取"""
    full_text = ""

    try:
        with pdfplumber.open(io.BytesIO(pdf_content)) as pdf:
            for page_num, page in enumerate(pdf.pages):
                # 尝试多种提取策略
                text = page.extract_text()
                if not text or len(text.strip()) < 50:
                    # 如果文本过少，尝试表格提取
                    tables = page.extract_tables()
                    for table in tables:
                        for row in table:
                            if row:
                                text += ' '.join([str(cell) for cell in row if cell]) + '\n'

                if text:
                    full_text += text + "\n\n"

    except Exception as e:
        print(f"文本提取警告: {e}")

    return full_text

def generate_professional_report(entities, context, filename, full_text):
    """生成专业报告"""
    report = []

    report.append("=" * 80)
    report.append(f"《{filename}》专业级实体识别分析报告")
    report.append("=" * 80)

    # 统计信息
    total_entities = sum(len(entity_list) for entity_list in entities.values())
    report.append(f"\n📊 精确识别统计:")
    report.append(f"   实体总数: {total_entities}")
    report.append(f"   👤 历史人物: {len(entities['person'])}")
    report.append(f"   🗺️ 地理地点: {len(entities['location'])}")
    report.append(f"   🏢 组织机构: {len(entities['organization'])}")
    report.append(f"   📅 历史事件: {len(entities['event'])}")
    report.append(f"   ⏰ 时间信息: {len(entities['time'])}")

    # 详细实体列表
    for entity_type, entity_list in entities.items():
        if entity_list:
            report.append(f"\n{'👤' if entity_type=='person' else '🗺️' if entity_type=='location' else '🏢' if entity_type=='organization' else '📅' if entity_type=='event' else '⏰'} {entity_type.upper()} ({len(entity_list)}个):")
            for i, entity in enumerate(entity_list[:25], 1):
                # 显示上下文
                context_info = context.get(entity_type, {}).get(entity, '')
                if context_info:
                    short_context = context_info[:60] + "..." if len(context_info) > 60 else context_info
                    report.append(f"   {i:2d}. {entity} | {short_context}")
                else:
                    report.append(f"   {i:2d}. {entity}")
            if len(entity_list) > 25:
                report.append(f"   ... 还有 {len(entity_list) - 25} 个{entity_type}")

    # 关键信息摘要
    report.append(f"\n🔍 关键信息摘要:")

    # 高频实体
    all_persons = entities['person']
    if all_persons:
        person_freq = Counter(all_persons)
        top_persons = person_freq.most_common(5)
        report.append(f"   核心人物: {', '.join([p[0] for p in top_persons])}")

    all_locations = entities['location']
    if all_locations:
        location_freq = Counter(all_locations)
        top_locations = location_freq.most_common(5)
        report.append(f"   主要地点: {', '.join([l[0] for l in top_locations])}")

    # 时间线
    if entities['time']:
        report.append(f"   时间线索: {', '.join(entities['time'][:8])}")

    # 置信度说明
    report.append(f"\n💡 识别置信度说明:")
    report.append("   ★★★ 专业词典匹配 - 最高置信度")
    report.append("   ★★☆ 规则模式匹配 - 高置信度")
    report.append("   ★☆☆ 统计模型识别 - 中等置信度")

    return "\n".join(report)

def main():
    print("请上传PDF文档...")
    uploaded = files.upload()

    if not uploaded:
        print("没有文件上传")
        return

    filename = list(uploaded.keys())[0]
    file_content = uploaded[filename]

    print(f"✓ 已上传: {filename}")
    print("正在启动专业级分析引擎...")

    # 提取文本
    full_text = extract_text_advanced(file_content)

    if not full_text.strip():
        print("❌ 无法提取有效文本")
        return

    print(f"✓ 文本提取完成: {len(full_text)} 字符")

    # 初始化专业NER
    ner = ProfessionalChineseNER()

    # 精确实体识别
    print("正在进行多维度精确识别...")

    entities = {
        'person': ner.precise_person_recognition(full_text),
        'location': ner.precise_location_recognition(full_text),
        'organization': ner.precise_organization_recognition(full_text),
        'event': ner.precise_event_recognition(full_text),
        'time': ner.time_recognition(full_text)
    }

    # 提取上下文
    context = ner.extract_with_context(full_text, entities)

    # 生成报告
    report = generate_professional_report(entities, context, filename, full_text)

    # 保存结果
    base_name = filename.replace('.pdf', '').replace('.PDF', '')

    # 保存专业报告
    report_filename = f"{base_name}_专业分析报告.txt"
    with open(report_filename, 'w', encoding='utf-8') as f:
        f.write(report)

    # 保存实体数据
    csv_data = []
    for entity_type, entity_list in entities.items():
        for entity in entity_list:
            csv_data.append({
                '实体类型': entity_type,
                '实体内容': entity,
                '上下文': context.get(entity_type, {}).get(entity, '')
            })

    if csv_data:
        df = pd.DataFrame(csv_data)
        csv_filename = f"{base_name}_实体数据.csv"
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

    print(f"\n✅ 专业分析完成!")
    print(f"   报告文件: {report_filename}")
    if csv_data:
        print(f"   数据文件: {csv_filename}")

    # 实时显示关键结果
    print(f"\n🎯 实时识别结果:")
    for entity_type, items in entities.items():
        if items:
            print(f"   {entity_type}: {len(items)}个 | 示例: {', '.join(items[:3])}")

if __name__ == "__main__":
    main()

ERROR: Could not find a version that satisfies the requirement fasthanlan (from versions: none)
ERROR: No matching distribution found for fasthanlan
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 34.5 MB/s eta 0:00:00
=== 专业级中文PDF信息精确提取系统 ===
请上传PDF文档...


Saving 《张隐韬烈士日记》连载(一)_王金风.pdf to 《张隐韬烈士日记》连载(一)_王金风.pdf
✓ 已上传: 《张隐韬烈士日记》连载(一)_王金风.pdf
正在启动专业级分析引擎...
✓ 文本提取完成: 13207 字符
正在进行多维度精确识别...

✅ 专业分析完成!
   报告文件: 《张隐韬烈士日记》连载(一)_王金风_专业分析报告.txt
   数据文件: 《张隐韬烈士日记》连载(一)_王金风_实体数据.csv

🎯 实时识别结果:
   person: 1914个 | 示例: 万人, 万佛, 万佛殿
   location: 52个 | 示例: 上海, 两省, 两色之旗
   organization: 25个 | 示例: 与民党, 中国共产党, 中央公园
   event: 17个 | 示例: 从事于革命, 从事于革命运动, 以为余之社会运动
   time: 7个 | 示例: 一九一七年, 一九一八年, 一九二一年


In [65]:
# 精准中文PDF信息提取系统 - 修复版
!pip install pdfplumber jieba pandas numpy requests -qq
!pip install paddlepaddle -qq
!pip install "paddleocr>=2.6.0" -qq

import pdfplumber
import re
import jieba
import jieba.posseg as pseg
import pandas as pd
import numpy as np
from collections import Counter
from google.colab import files
import io
import os
from PIL import Image
import cv2

print("=== 精准中文PDF信息提取系统 ===")

# 高级初始化jieba
jieba.initialize()

# 专业历史文献词典 - 扩展版
PROFESSIONAL_DICT = {
    'person': [
        '张隐韬', '王金风', '李大钊', '毛泽东', '周恩来', '蒋介石', '孙中山', '袁世凯',
        '陈独秀', '胡适', '鲁迅', '梁启超', '蔡元培', '章太炎', '朱德', '刘少奇', '邓小平',
        '彭德怀', '贺龙', '刘伯承', '陈毅', '林彪', '宋庆龄', '冯玉祥', '张学良', '杨虎城',
        '瞿秋白', '张作霖', '吴佩孚', '段祺瑞', '曹锟', '黎元洪', '徐世昌', '汪精卫', '胡汉民'
    ],
    'location': [
        '南皮县', '河北省', '天津市', '北京市', '上海市', '广州市', '武汉市', '南京市',
        '重庆市', '西安市', '延安', '井冈山', '瑞金', '遵义', '庐山', '黄埔', '北平', '南京',
        '沈阳', '长春', '哈尔滨', '济南', '青岛', '成都', '长沙', '合肥', '郑州', '石家庄',
        '太原', '呼和浩特', '杭州', '宁波', '福州', '厦门', '南宁', '海口', '贵阳', '昆明',
        '兰州', '西宁', '银川', '乌鲁木齐', '拉萨', '香港', '澳门', '台湾'
    ],
    'organization': [
        '中国共产党', '国民党', '黄埔军校', '国民军', '八路军', '新四军', '解放军',
        '中央政府', '国务院', '政治局', '中央委员会', '共青团', '工会', '农会',
        '国民政府', '军事委员会', '中共中央', '全国人大', '全国政协', '中央军委'
    ],
    'event': [
        '五四运动', '北伐战争', '南昌起义', '秋收起义', '抗日战争', '解放战争', '土地改革',
        '文化大革命', '改革开放', '第一次国共合作', '第二次国共合作', '西安事变',
        '卢沟桥事变', '百团大战', '辽沈战役', '淮海战役', '平津战役', '遵义会议',
        '延安整风', '重庆谈判', '抗美援朝', '大跃进', '人民公社化运动'
    ]
}

# 加载专业词典到jieba
for category, words in PROFESSIONAL_DICT.items():
    for word in words:
        jieba.add_word(word, freq=1000000)

class PrecisionChineseNER:
    def __init__(self):
        self.surname_set = self._load_comprehensive_surnames()
        self.location_keywords = self._load_location_keywords()
        self.event_keywords = self._load_event_keywords()

    def _load_comprehensive_surnames(self):
        """加载完整姓氏库"""
        surnames = set()
        # 现代常见姓氏（前200个）
        common_surnames = '''
            李王张刘陈杨赵黄周吴徐孙胡朱高林何郭马罗梁宋郑谢韩唐冯于董萧程曹袁邓许傅沈曾彭吕苏卢蒋蔡贾丁魏薛叶阎余潘杜戴夏钟汪田任姜范方石姚谭廖邹熊金陆郝孔白崔康毛邱秦江史顾侯邵孟龙万段漕钱汤尹黎易常武乔贺赖龚文
            庞樊兰殷施陶洪翟安颜倪严牛温芦季俞章鲁葛伍韦申尤毕聂丛焦向柳邢路岳齐沿梅莫庄辛管祝左涂谷祁时舒耿牟卜路詹关苗凌费纪靳盛童欧甄项曲成游阳裴席卫查屈鲍位覃霍翁隋植甘景薄单包司柏宁柯阮桂闵欧阳解强丛华
            全佟应臧闵慕连茹习宦艾鱼容向古易慎戈廖庾终暨居衡步都耿满弘匡国文寇广禄阙东欧殳沃利蔚越夔隆师巩厍聂晁勾敖融冷訾辛阚那简饶空曾毋沙乜养鞠须丰巢关蒯相查后荆红游竺权逯盖益桓公万俟司马上官欧阳夏侯诸葛
        '''
        for char in common_surnames:
            if char.strip():
                surnames.add(char)
        return surnames

    def _load_location_keywords(self):
        """加载地理位置关键词"""
        provinces = ['河北','山西','辽宁','吉林','黑龙江','江苏','浙江','安徽','福建','江西',
                    '山东','河南','湖北','湖南','广东','海南','四川','贵州','云南','陕西',
                    '甘肃','青海','台湾','内蒙古','广西','西藏','宁夏','新疆']

        major_cities = ['北京','天津','上海','重庆','石家庄','太原','沈阳','长春','哈尔滨',
                       '南京','杭州','合肥','福州','南昌','济南','郑州','武汉','长沙','广州',
                       '南宁','海口','成都','贵阳','昆明','西安','兰州','西宁','台北']

        regions = ['华北','华东','华南','华中','西北','西南','东北','京津冀','长三角','珠三角']

        return set(provinces + major_cities + regions)

    def _load_event_keywords(self):
        """加载事件关键词"""
        return {
            '运动': ['五四运动', '整风运动', '土地改革运动'],
            '起义': ['南昌起义', '秋收起义', '广州起义'],
            '战争': ['北伐战争', '抗日战争', '解放战争'],
            '战役': ['辽沈战役', '淮海战役', '平津战役', '百团大战'],
            '事件': ['西安事变', '卢沟桥事变', '重庆谈判'],
            '会议': ['遵义会议', '古田会议', '八七会议']
        }

    def precision_person_extraction(self, text):
        """精准人物提取"""
        persons = set()

        # 1. 专业词典匹配（最高优先级）
        for person in PROFESSIONAL_DICT['person']:
            if person in text:
                persons.add(person)

        # 2. 基于姓氏的精确匹配
        for surname in list(self.surname_set)[:100]:  # 使用前100个常见姓氏
            # 匹配2-3字中文名
            patterns = [
                surname + r'[\u4e00-\u9fa5]{1}',  # 姓+1字
                surname + r'[\u4e00-\u9fa5]{2}',  # 姓+2字
            ]

            for pattern in patterns:
                matches = re.findall(pattern, text)
                for match in matches:
                    if 2 <= len(match) <= 3:  # 限制合理的姓名长度
                        # 进一步验证：不能是常见词语
                        common_words = ['老师', '先生', '同志', '领导', '干部', '党员']
                        if not any(word in match for word in common_words):
                            persons.add(match)

        # 3. jieba词性标注 + 姓氏验证
        words = pseg.cut(text)
        for word, flag in words:
            if flag == 'nr' and 2 <= len(word) <= 4:
                # 验证是否以常见姓氏开头
                if any(word.startswith(surname) for surname in list(self.surname_set)[:50]):
                    persons.add(word)

        return sorted(list(persons))

    def precision_location_extraction(self, text):
        """精准地点提取"""
        locations = set()

        # 1. 专业词典匹配
        for location in PROFESSIONAL_DICT['location']:
            if location in text:
                locations.add(location)

        # 2. 行政区划模式匹配
        administrative_patterns = [
            r'([\u4e00-\u9fa5]{2,4}省)',
            r'([\u4e00-\u9fa5]{1,6}市)',
            r'([\u4e00-\u9fa5]{1,6}县)',
            r'([\u4e00-\u9fa5]{1,6}区)',
            r'([\u4e00-\u9fa5]{1,6}镇)',
            r'([\u4e00-\u9fa5]{1,6}乡)',
            r'([\u4e00-\u9fa5]{1,6}村)'
        ]

        for pattern in administrative_patterns:
            matches = re.findall(pattern, text)
            locations.update(matches)

        # 3. 特定地理关键词
        for keyword in self.location_keywords:
            if keyword in text:
                locations.add(keyword)

        # 4. jieba识别
        words = pseg.cut(text)
        for word, flag in words:
            if flag == 'ns' and len(word) >= 2:
                locations.add(word)

        return sorted(list(locations))

    def precision_organization_extraction(self, text):
        """精准组织提取"""
        organizations = set()

        # 1. 专业词典匹配
        for org in PROFESSIONAL_DICT['organization']:
            if org in text:
                organizations.add(org)

        # 2. 组织后缀匹配
        org_suffixes = [
            '党', '政府', '委员会', '部队', '军队', '学校', '大学', '学院',
            '机关', '部门', '局', '办公室', '小组', '兵团', '军区'
        ]

        for suffix in org_suffixes:
            pattern = r'[\u4e00-\u9fa5]{1,8}' + suffix
            matches = re.findall(pattern, text)
            organizations.update(matches)

        # 3. jieba识别
        words = pseg.cut(text)
        for word, flag in words:
            if flag == 'nt' and len(word) >= 2:
                organizations.add(word)

        return sorted(list(organizations))

    def precision_event_extraction(self, text):
        """精准事件提取"""
        events = set()

        # 1. 专业词典匹配
        for event in PROFESSIONAL_DICT['event']:
            if event in text:
                events.add(event)

        # 2. 事件模式匹配
        event_patterns = [
            r'[\u4e00-\u9fa5]{1,6}运动',
            r'[\u4e00-\u9fa5]{1,6}起义',
            r'[\u4e00-\u9fa5]{1,6}战争',
            r'[\u4e00-\u9fa5]{1,6}战役',
            r'[\u4e00-\u9fa5]{1,6}事件',
            r'[\u4e00-\u9fa5]{1,6}改革',
            r'[\u4e00-\u9fa5]{1,6}革命',
            r'[\u4e00-\u9fa5]{1,6}会议'
        ]

        for pattern in event_patterns:
            matches = re.findall(pattern, text)
            events.update(matches)

        # 3. 时间+事件组合
        time_event_pattern = r'(\d{1,4}年\S{1,8}?(?:运动|起义|战争|战役|事件|改革|会议))'
        matches = re.findall(time_event_pattern, text)
        events.update(matches)

        return sorted(list(events))

    def time_extraction(self, text):
        """时间信息提取"""
        times = set()

        # 多种时间格式
        time_patterns = [
            r'\d{1,4}年\d{1,2}月\d{1,2}日',
            r'\d{1,4}年\d{1,2}月',
            r'\d{1,4}年',
            r'\d{1,2}月\d{1,2}日',
            r'[一二三四五六七八九十]+年',
            r'(?:民国|公元|光绪|宣统|康熙|乾隆)\d{1,4}年',
            r'\d{1,4}年代'
        ]

        for pattern in time_patterns:
            matches = re.findall(pattern, text)
            times.update(matches)

        return sorted(list(times))

    def extract_entity_context(self, text, entities, context_window=100):
        """提取实体上下文"""
        context_info = {}

        for entity_type, entity_list in entities.items():
            context_info[entity_type] = {}
            for entity in entity_list[:15]:  # 每个类型取前15个提取上下文
                # 找到实体位置
                start_pos = text.find(entity)
                if start_pos != -1:
                    # 提取上下文
                    start = max(0, start_pos - context_window)
                    end = min(len(text), start_pos + len(entity) + context_window)
                    context = text[start:end].replace('\n', ' ')
                    context_info[entity_type][entity] = context

        return context_info

def extract_text_with_fallback(pdf_content):
    """带降级策略的文本提取"""
    full_text = ""

    try:
        with pdfplumber.open(io.BytesIO(pdf_content)) as pdf:
            for page_num, page in enumerate(pdf.pages):
                # 主要文本提取
                text = page.extract_text()

                # 如果主要文本过少，尝试提取表格
                if not text or len(text.strip()) < 30:
                    tables = page.extract_tables()
                    for table in tables:
                        if table:
                            for row in table:
                                if row:
                                    row_text = ' '.join([str(cell) for cell in row if cell])
                                    if row_text:
                                        text += row_text + '\n'

                if text:
                    full_text += f"第{page_num+1}页:\n{text}\n\n"

    except Exception as e:
        print(f"文本提取警告: {e}")

    return full_text

def generate_precision_report(entities, context, filename, full_text):
    """生成精准分析报告"""
    report = []

    report.append("=" * 80)
    report.append(f"《{filename}》精准实体识别分析报告")
    report.append("=" * 80)

    # 统计信息
    total_entities = sum(len(entity_list) for entity_list in entities.values())
    report.append(f"\n📊 精确识别统计:")
    report.append(f"   实体总数: {total_entities}")
    report.append(f"   👤 历史人物: {len(entities['person'])}")
    report.append(f"   🗺️ 地理地点: {len(entities['location'])}")
    report.append(f"   🏢 组织机构: {len(entities['organization'])}")
    report.append(f"   📅 历史事件: {len(entities['event'])}")
    report.append(f"   ⏰ 时间信息: {len(entities['time'])}")

    # 详细实体列表（带上下文）
    for entity_type, entity_list in entities.items():
        if entity_list:
            icon = {
                'person': '👤', 'location': '🗺️', 'organization': '🏢',
                'event': '📅', 'time': '⏰'
            }.get(entity_type, '📌')

            report.append(f"\n{icon} {entity_type.upper()} ({len(entity_list)}个):")

            for i, entity in enumerate(entity_list[:20], 1):
                # 显示上下文信息
                ctx = context.get(entity_type, {}).get(entity, '')
                if ctx:
                    # 清理上下文文本
                    clean_ctx = re.sub(r'\s+', ' ', ctx).strip()
                    if len(clean_ctx) > 80:
                        clean_ctx = clean_ctx[:80] + "..."
                    report.append(f"   {i:2d}. {entity}")
                    report.append(f"       上下文: {clean_ctx}")
                else:
                    report.append(f"   {i:2d}. {entity}")

            if len(entity_list) > 20:
                report.append(f"   ... 还有 {len(entity_list) - 20} 个{entity_type}")

    # 关键信息分析
    report.append(f"\n🔍 关键信息分析:")

    # 核心人物分析
    if entities['person']:
        person_counter = Counter(entities['person'])
        top_persons = person_counter.most_common(3)
        report.append(f"   核心人物: {', '.join([p[0] for p in top_persons])}")

    # 主要地点分析
    if entities['location']:
        location_counter = Counter(entities['location'])
        top_locations = location_counter.most_common(3)
        report.append(f"   主要地点: {', '.join([l[0] for l in top_locations])}")

    # 时间线分析
    if entities['time']:
        report.append(f"   时间线索: {', '.join(entities['time'][:5])}")

    # 置信度说明
    report.append(f"\n💡 识别精度说明:")
    report.append("   ✅ 专业词典匹配 - 接近100%准确率")
    report.append("   ✅ 规则模式匹配 - 高准确率")
    report.append("   ⚠️  统计模型识别 - 需人工核对")

    return "\n".join(report)

def main():
    print("请上传PDF文档...")
    uploaded = files.upload()

    if not uploaded:
        print("没有文件上传")
        return

    filename = list(uploaded.keys())[0]
    file_content = uploaded[filename]

    print(f"✓ 已上传: {filename}")
    print("正在启动精准分析引擎...")

    # 提取文本
    full_text = extract_text_with_fallback(file_content)

    if not full_text.strip():
        print("❌ 无法提取有效文本，请检查PDF格式")
        return

    text_length = len(full_text)
    print(f"✓ 文本提取完成: {text_length} 字符")

    if text_length < 100:
        print("⚠️  提取文本较少，可能是扫描版PDF")

    # 初始化精准NER
    ner = PrecisionChineseNER()

    print("正在进行多维度精准识别...")

    # 执行精准实体识别
    entities = {
        'person': ner.precision_person_extraction(full_text),
        'location': ner.precision_location_extraction(full_text),
        'organization': ner.precision_organization_extraction(full_text),
        'event': ner.precision_event_extraction(full_text),
        'time': ner.time_extraction(full_text)
    }

    # 提取上下文信息
    context = ner.extract_entity_context(full_text, entities)

    # 生成报告
    report = generate_precision_report(entities, context, filename, full_text)

    # 保存结果
    base_name = filename.replace('.pdf', '').replace('.PDF', '')

    # 保存精准报告
    report_filename = f"{base_name}_精准分析报告.txt"
    with open(report_filename, 'w', encoding='utf-8') as f:
        f.write(report)

    # 保存实体数据到CSV
    csv_data = []
    for entity_type, entity_list in entities.items():
        for entity in entity_list:
            csv_data.append({
                '实体类型': entity_type,
                '实体内容': entity,
                '上下文': context.get(entity_type, {}).get(entity, ''),
                '置信度': '高' if entity in PROFESSIONAL_DICT.get(entity_type, []) else '中'
            })

    if csv_data:
        df = pd.DataFrame(csv_data)
        csv_filename = f"{base_name}_实体数据.csv"
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

    print(f"\n✅ 精准分析完成!")
    print(f"📄 报告文件: {report_filename}")
    if csv_data:
        print(f"📊 数据文件: {csv_filename}")

    # 实时显示关键结果
    print(f"\n🎯 核心识别结果:")
    for entity_type, items in entities.items():
        if items:
            sample = items[:3]
            print(f"   {entity_type}: {len(items)}个 | 示例: {', '.join(sample)}")

# 运行主程序
if __name__ == "__main__":
    main()


=== 精准中文PDF信息提取系统 ===
请上传PDF文档...


Saving 《张隐韬烈士日记》连载(一)_王金风.pdf to 《张隐韬烈士日记》连载(一)_王金风 (1).pdf
✓ 已上传: 《张隐韬烈士日记》连载(一)_王金风 (1).pdf
正在启动精准分析引擎...
✓ 文本提取完成: 13252 字符
正在进行多维度精准识别...

✅ 精准分析完成!
📄 报告文件: 《张隐韬烈士日记》连载(一)_王金风 (1)_精准分析报告.txt
📊 数据文件: 《张隐韬烈士日记》连载(一)_王金风 (1)_实体数据.csv

🎯 核心识别结果:
   person: 594个 | 示例: 万人, 万佛, 万佛殿
   location: 137个 | 示例: 七绝, 上台, 上海
   organization: 40个 | 示例: 与民党, 与该弥兵团, 且休绕越过局
   event: 17个 | 示例: 从事于革命, 从事于革命运动, 以为余之社会运动
   time: 7个 | 示例: 一九一七年, 一九一八年, 一九二一年
